In [2]:
#resnet15=0.78
#seresnet15=0.78,0.8
#FFN=0.81

In [1]:
# importing necessary library
import h5py
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from tqdm import tqdm
import pickle
import timm
from sklearn.metrics import roc_auc_score
# importing torch_libraries
import torch
from torch import nn
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
#Creating Train and Validation Dataset and DataLoader for Training of the model
with h5py.File("/kaggle/input/e2e-common-task/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5", "r") as f:
    # print("Keys: ", list(f.keys()))
    dataset1 = f["X"]
    dataset2 = f['y']
    image_electron = dataset1[:]
    label_electron= dataset2[:]

with h5py.File("/kaggle/input/e2e-common-task/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5", "r") as f:
    # print("Keys: ", list(f.keys()))
    dataset1 = f["X"]
    dataset2 = f['y']
    image_photon = dataset1[:]
    label_photon= dataset2[:]

plt.imshow(image_electron[0][:,:,1])
plt.axis("off")  # Hide axis
plt.show()

image_val=[]
label_val=[]
image_train=[]
label_train=[]
for i in range(249000):
    if i<49800:
        image_val.append(image_electron[i])
        label_val.append(label_electron[i])
        image_val.append(image_photon[i])
        label_val.append(label_photon[i])
    else:
        image_train.append(image_electron[i])
        label_train.append(label_electron[i])
        image_train.append(image_photon[i])
        label_train.append(label_photon[i])

class custom_dataset(Dataset):
    def __init__(self,image_list,label_list):
        super().__init__()
        self.image_list=image_list
        self.label_list=label_list
    def __len__(self):
        return len(self.label_list)
    def __getitem__(self,ind):
        image=torch.from_numpy(self.image_list[ind])
        image=torch.permute(image,(2,0,1))[0].unsqueeze(0)
        label=torch.tensor(self.label_list[ind],dtype=torch.float)
        return image,label

val_dataset=custom_dataset(image_val,label_val)
train_dataset=custom_dataset(image_train,label_train)

val_dataloader=DataLoader(val_dataset,batch_size=64,shuffle=True,num_workers=os.cpu_count())
train_dataloader=DataLoader(train_dataset,batch_size=64,shuffle=True,num_workers=os.cpu_count())

In [12]:
#RsNet15 class which imaplements models which have Parameters comparable to Resnet15
class ResNet15(nn.Module):
    def __init__(self, num_classes=1000):
        super(ResNet15, self).__init__()
        
        # Load ResNet-18 from timm
        resnet18 = timm.create_model('seresnet18', pretrained=False, in_chans=1)
        
        # Keep initial layers
        self.conv1 = resnet18.conv1
        self.bn1 = resnet18.bn1
        self.act1 = resnet18.act1  # ReLU
        self.maxpool = resnet18.maxpool
        
        # Modify residual layers (removing one block each from layer3 and layer4)
        self.layer1 = resnet18.layer1  # 2 blocks
        self.layer2 = resnet18.layer2  # 2 blocks
        self.layer3 = nn.Sequential(*list(resnet18.layer3.children())[:1])  # 1 block
        self.layer4 = nn.Sequential(*list(resnet18.layer4.children())[:1])  # 1 block
        
        # Adaptive pooling & FC layer
        self.global_pool = resnet18.global_pool
        self.fc = nn.Linear(512, num_classes)  # Adjust output features
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.global_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

#Feed Forward Neural Network For experimentation
class ScaledFNN(nn.Module):
    def __init__(self, input_size=32*32*1, hidden_sizes=[2048, 1024, 512, 256, 128], num_classes=1):
        super(ScaledFNN, self).__init__()
        
        self.flatten = nn.Flatten()
        self.fc_layers = nn.Sequential(
            nn.Linear(input_size, hidden_sizes[0]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[1], hidden_sizes[2]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[2], hidden_sizes[3]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[3], hidden_sizes[4]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[4], num_classes)
        )

    def forward(self, x):
        x = self.flatten(x)  # Convert (32,32,1) → (1024)
        x = self.fc_layers(x)
        return x

# Instantiate model
model = ResNet15(1)

In [22]:
#Define Loss and Optimizer
model = pickle.load(open("/kaggle/working/best_model.sav", "rb")).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.NAdam(model.parameters(), lr=0.000004)
# model

In [24]:
num_epochs=10
best=10000.0
# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for images, labels in tqdm(train_dataloader):
        images, labels = images.to(device), labels.to(device).unsqueeze(-1)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= len(train_dataloader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    pred=[]
    label=[]
    with torch.no_grad():
        for images, labels in tqdm(val_dataloader):
            images, labels = images.to(device), labels.to(device).unsqueeze(-1)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            pred.append(torch.nn.Sigmoid()(outputs).cpu().numpy())
            label.append(labels.cpu().numpy())
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    predictions=np.concatenate(pred)
    label=np.concatenate(label)
    score =  roc_auc_score(label, predictions, multi_class="ovr")
    val_loss /= len(val_dataloader)
    val_acc = 100 * correct / (len(val_dataloader)*32)
    if val_loss<best:
        best=val_loss
        pickle.dump(model, open('/kaggle/working/best_model.sav', 'wb'))
        
    # Print results
    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val roc_auc: {score:.2f}")

100%|██████████| 1557/1557 [00:10<00:00, 149.89it/s]


Epoch [1/10] | Train Loss: 0.5339 | Val Loss: 0.5410 | Val roc_auc: 0.80


 11%|█         | 658/6225 [00:11<01:35, 58.29it/s]


KeyboardInterrupt: 